In [2]:
#!pip install scikit-optimize

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 2.7 MB/s eta 0:00:00


In [4]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import randint, uniform
import time
import matplotlib.pyplot as plt
from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical

# Configurar el formato de visualización para números flotantes
pd.set_option('display.float_format', '{:.4f}'.format)

# Generar un conjunto de datos sintético
X, y = make_regression(n_samples=1000, n_features=20, n_informative=10,
                       noise=0.5, random_state=42)

# Dividir los datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Definir el modelo base
base_model = RandomForestRegressor(random_state=42)

# Definir los espacios de búsqueda para cada método
# Para GridSearchCV - espacio discreto
param_grid = {
    'n_estimators': [10, 50, 100],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Para RandomizedSearchCV - distribuciones
param_distributions = {
    'n_estimators': randint(10, 100),
    'max_depth': randint(5, 30),
    'min_samples_split': randint(2, 11),
    'min_samples_leaf': randint(1, 5)
}

# Para BayesSearchCV - espacio de búsqueda
search_spaces = {
    'n_estimators': Integer(10, 100),
    'max_depth': Integer(5, 30),
    'min_samples_split': Integer(2, 11),
    'min_samples_leaf': Integer(1, 5)
}

# Función para evaluar y registrar el rendimiento
def evaluate_search(search, name, X_train, y_train, X_test, y_test):
    # Medir tiempo de inicio
    start_time = time.time()

    # Ajustar el modelo
    search.fit(X_train, y_train)

    # Medir tiempo de finalización
    end_time = time.time()
    elapsed_time = end_time - start_time

    # Obtener el mejor modelo
    best_model = search.best_estimator_

    # Predecir con los datos de prueba
    y_pred = best_model.predict(X_test)

    # Calcular métricas
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    # Número de modelos evaluados
    n_models = len(search.cv_results_['mean_test_score'])

    # Tiempo promedio por modelo
    avg_time_per_model = elapsed_time / n_models

    return {
        'Método': name,
        'Mejor R²': r2,
        'MSE': mse,
        'Tiempo total (s)': elapsed_time,
        'Modelos evaluados': n_models,
        'Tiempo por modelo (s)': avg_time_per_model,
        'Mejores parámetros': search.best_params_
    }

# Configurar cada método de búsqueda
# Limitamos el número de iteraciones para que la comparación sea justa
n_iter = 20  # Número de iteraciones para RandomizedSearchCV y BayesSearchCV

# 1. GridSearchCV
grid_search = GridSearchCV(
    estimator=base_model,
    param_grid=param_grid,
    cv=5,  # Validación cruzada de 3 pliegues para ahorrar tiempo
    scoring='neg_mean_squared_error',
    n_jobs=-1
)

# 2. RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=base_model,
    param_distributions=param_distributions,
    n_iter=n_iter,
    cv=5,
    scoring='neg_mean_squared_error',
    random_state=42,
    n_jobs=-1
)

# 3. BayesSearchCV
try:
    bayes_search = BayesSearchCV(
        estimator=base_model,
        search_spaces=search_spaces,
        n_iter=n_iter,
        cv=5,
        scoring='neg_mean_squared_error',
        random_state=42,
        n_jobs=-1
    )
    has_skopt = True
except:
    print("scikit-optimize no está instalado. Omitiendo BayesSearchCV.")
    has_skopt = False

# Ejecutar y evaluar cada método
results = []

# GridSearchCV
print("Ejecutando GridSearchCV...")
grid_result = evaluate_search(grid_search, "GridSearchCV", X_train, y_train, X_test, y_test)
results.append(grid_result)

# RandomizedSearchCV
print("Ejecutando RandomizedSearchCV...")
random_result = evaluate_search(random_search, "RandomizedSearchCV", X_train, y_train, X_test, y_test)
results.append(random_result)

# BayesSearchCV (si está disponible)
if has_skopt:
    print("Ejecutando BayesSearchCV...")
    bayes_result = evaluate_search(bayes_search, "BayesSearchCV", X_train, y_train, X_test, y_test)
    results.append(bayes_result)
else:
    print("Omitiendo BayesSearchCV debido a que scikit-optimize no está instalado.")

# Crear un DataFrame con los resultados
results_df = pd.DataFrame(results)
print("\nComparación de métodos de búsqueda de hiperparámetros:")
print(results_df[['Método', 'Mejor R²', 'MSE', 'Tiempo total (s)', 'Modelos evaluados', 'Tiempo por modelo (s)']])

print("\nMejores parámetros encontrados:")
for result in results:
    print(f"\n{result['Método']}:")
    for param, value in result['Mejores parámetros'].items():
        print(f"  {param}: {value}")

# Visualizar resultados
plt.figure(figsize=(14, 10))

# Gráfico de tiempo total
plt.subplot(2, 2, 1)
plt.bar(results_df['Método'], results_df['Tiempo total (s)'])
plt.title('Tiempo total de ejecución (s)')
plt.xticks(rotation=45)
plt.tight_layout()

# Gráfico de R²
plt.subplot(2, 2, 2)
plt.bar(results_df['Método'], results_df['Mejor R²'])
plt.title('Mejor R² encontrado')
plt.xticks(rotation=45)
plt.tight_layout()

# Gráfico de modelos evaluados
plt.subplot(2, 2, 3)
plt.bar(results_df['Método'], results_df['Modelos evaluados'])
plt.title('Número de modelos evaluados')
plt.xticks(rotation=45)
plt.tight_layout()

# Gráfico de tiempo por modelo
plt.subplot(2, 2, 4)
plt.bar(results_df['Método'], results_df['Tiempo por modelo (s)'])
plt.title('Tiempo promedio por modelo (s)')
plt.xticks(rotation=45)
plt.tight_layout()

plt.savefig('comparacion_metodos_busqueda.png')
plt.close()

# Calcular la eficiencia (R² / tiempo)
results_df['Eficiencia (R²/tiempo)'] = results_df['Mejor R²'] / results_df['Tiempo total (s)']

plt.figure(figsize=(10, 6))
plt.bar(results_df['Método'], results_df['Eficiencia (R²/tiempo)'])
plt.title('Eficiencia (R² por segundo)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('eficiencia_metodos_busqueda.png')
plt.close()

# Mostrar la tabla de eficiencia
print("\nEficiencia de los métodos:")
print(results_df[['Método', 'Eficiencia (R²/tiempo)']])

Ejecutando GridSearchCV...
Ejecutando RandomizedSearchCV...
Ejecutando BayesSearchCV...

Comparación de métodos de búsqueda de hiperparámetros:
               Método  Mejor R²       MSE  Tiempo total (s)  \
0        GridSearchCV    0.8153 7149.5668          157.5338   
1  RandomizedSearchCV    0.8157 7135.4248           36.8138   
2       BayesSearchCV    0.8158 7129.0141           73.2166   

   Modelos evaluados  Tiempo por modelo (s)  
0                 81                 1.9449  
1                 20                 1.8407  
2                 20                 3.6608  

Mejores parámetros encontrados:

GridSearchCV:
  max_depth: None
  min_samples_leaf: 2
  min_samples_split: 2
  n_estimators: 100

RandomizedSearchCV:
  max_depth: 22
  min_samples_leaf: 2
  min_samples_split: 3
  n_estimators: 83

BayesSearchCV:
  max_depth: 24
  min_samples_leaf: 2
  min_samples_split: 2
  n_estimators: 90

Eficiencia de los métodos:
               Método  Eficiencia (R²/tiempo)
0        GridSear